<a href="https://colab.research.google.com/github/Scor999/Introduction-to-neural-networks/blob/main/dz6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, concatenate, Conv2DTranspose
from keras.optimizers import Adam
from keras.utils import to_categorical


In [42]:
# Загрузка и предобработка данных
def download_data(path):
    data = []
    for path_image in sorted(os.listdir(path=path)):
        image = Image.open(path + path_image) # Открываем изображение.
        data.append(np.array(image)) # Загружаем пиксели.
    return data

def resize(input_image, input_mask):
    input_image = tf.image.resize(input_image, (128, 128), method="nearest")
    input_mask = tf.image.resize(input_mask, (128, 128), method="nearest")
    return input_image, input_mask

In [43]:
X_train = download_data(r"train/images/")
Y_train = download_data(r"train/masks/")
X_test = download_data(r"test/images/")
Y_test = download_data(r"test/masks/")


In [44]:
X_train, Y_train = resize(X_train, Y_train)
X_test, Y_test = resize(X_test, Y_test)

In [46]:
# Определение палитры цветов
palette = {
    (60, 16, 152): 0,    # Building
    (132, 41, 246): 1,   # Land
    (110, 193, 228): 2,  # Road
    (254, 221, 58): 3,   # Vegetation
    (226, 169, 41): 4,   # Water
    (155, 155, 155): 5   # Unlabeled
}

In [47]:
# Инвертированная палитра
invert_palette = {v: k for k, v in palette.items()}

def convert_from_color(arr_3d, palette=invert_palette):
    """ RGB-color encoding to grayscale labels """
    arr_2d = np.zeros((arr_3d.shape[0], arr_3d.shape[1]), dtype=np.int8)
    for c, i in palette.items():
        m = np.all(arr_3d == np.array(c).reshape(1, 1, 3), axis=2)
        arr_2d[m] = i

    arr_2d = arr_2d.tolist()
    for i in range(len(arr_2d)):
        for j in range(len(arr_2d[0])):
            label = [0, 0, 0, 0, 0, 0]
            label[arr_2d[i][j]] = 1
            arr_2d[i][j] = label
    arr_2d = np.array(arr_2d)

    return arr_2d

In [48]:
# Преобразование меток данных в one-hot encoded формат с использованием палитры цветов
Y_train_pred = []
for i in range(len(Y_train)):
    Y_train_pred.append(convert_from_color(Y_train[i][:, :, :3], palette))
Y_train_pred = np.array(Y_train_pred)

Y_test_pred = []
for i in range(len(Y_test)):
    Y_test_pred.append(convert_from_color(Y_test[i][:, :, :3], palette))
Y_test_pred = np.array(Y_test_pred)

In [49]:
# Определение U-Net модели
def conv_block(inputs, filters, kernel_size=(3, 3), activation='relu', padding='same'):
    conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(inputs)
    conv = Conv2D(filters, kernel_size, activation=activation, padding=padding)(conv)
    return conv

In [50]:
def build_unet_model(input_shape, num_classes):
    inputs = Input(input_shape)

    # Encoder
    conv1 = conv_block(inputs, 64)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = conv_block(pool1, 128)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = conv_block(pool2, 256)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = conv_block(pool3, 512)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = conv_block(pool4, 1024)

    # Decoder
    up6 = Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(conv5)
    up6 = concatenate([up6, conv4], axis=3)
    conv6 = conv_block(up6, 512)

    up7 = Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(conv6)
    up7 = concatenate([up7, conv3], axis=3)
    conv7 = conv_block(up7, 256)

    up8 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv7)
    up8 = concatenate([up8, conv2], axis=3)
    conv8 = conv_block(up8, 128)

    up9 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv8)
    up9 = concatenate([up9, conv1], axis=3)
    conv9 = conv_block(up9, 64)

    outputs = Conv2D(num_classes, (1, 1), activation='softmax')(conv9)

    model = Model(inputs=[inputs], outputs=[outputs])
    return model

In [74]:
# Определение архитектуры модели и компиляция
input_shape = (128, 128, 3)
num_classes = 6  # Предполагается 6 классов для сегментации

unet_model = build_unet_model(input_shape, num_classes)
unet_model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [76]:
# Обучение модели
history = unet_model.fit(np.array(X_train)/255, Y_train_pred, epochs=5, validation_data=(np.array(X_test)/255, Y_test_pred))

Epoch 1/5
1/1 [==============================] - 12s 12s/step - loss: 1.1047 - accuracy: 0.6007 - val_loss: 1.1037 - val_accuracy: 0.5855
Epoch 2/5
1/1 [==============================] - 13s 13s/step - loss: 1.0986 - accuracy: 0.6007 - val_loss: 1.1002 - val_accuracy: 0.5855
Epoch 3/5
1/1 [==============================] - 12s 12s/step - loss: 1.0819 - accuracy: 0.6007 - val_loss: 1.0967 - val_accuracy: 0.5855
Epoch 4/5
1/1 [==============================] - 12s 12s/step - loss: 1.0638 - accuracy: 0.6007 - val_loss: 1.0990 - val_accuracy: 0.5855
Epoch 5/5
1/1 [==============================] - 12s 12s/step - loss: 1.0543 - accuracy: 0.6007 - val_loss: 1.1096 - val_accuracy: 0.5855


In [ ]:
out = unet_model.predict(X_test_pred, batch_size=1)

I = 0 # номер картинки после обработки нейронной сетью
plt.imshow(np.array(X_test)[I])
plt.show()
plt.imshow(np.array(Y_test)[I])
plt.show()
plt.imshow(convert_to_color(np.argmax(out[I], axis=-1)))
plt.show()

In [71]:
from keras.optimizers import SGD

# Определение архитектуры модели и компиляция
input_shape = (128, 128, 3)
num_classes = 6  # Предполагается 6 классов для сегментации

unet_model = build_unet_model(input_shape, num_classes)

# Использование оптимизатора SGD с параметрами по умолчанию
optimizer = SGD(lr=0.01, momentum=0.9)  # Установка скорости обучения и параметра момента

unet_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
history = unet_model.fit(np.array(X_train)/255, Y_train_pred, epochs=10, validation_data=(np.array(X_test)/255, Y_test_pred))

Epoch 1/10
1/1 [==============================] - 15s 15s/step - loss: 1.7873 - accuracy: 0.0253 - val_loss: 1.7704 - val_accuracy: 0.1935
Epoch 2/10
1/1 [==============================] - 11s 11s/step - loss: 1.7681 - accuracy: 0.1988 - val_loss: 1.7338 - val_accuracy: 0.5722
Epoch 3/10
1/1 [==============================] - 12s 12s/step - loss: 1.7315 - accuracy: 0.5843 - val_loss: 1.6910 - val_accuracy: 0.5854
Epoch 4/10
1/1 [==============================] - 12s 12s/step - loss: 1.6885 - accuracy: 0.6007 - val_loss: 1.6450 - val_accuracy: 0.5855
Epoch 5/10
1/1 [==============================] - 12s 12s/step - loss: 1.6422 - accuracy: 0.6007 - val_loss: 1.5787 - val_accuracy: 0.5855
Epoch 6/10
1/1 [==============================] - 12s 12s/step - loss: 1.5755 - accuracy: 0.6007 - val_loss: 1.4885 - val_accuracy: 0.5855
Epoch 7/10
1/1 [==============================] - 12s 12s/step - loss: 1.4853 - accuracy: 0.6007 - val_loss: 1.3756 - val_accuracy: 0.5855
Epoch 8/10
1/1 [===========

In [73]:
from keras.optimizers import Adagrad

# Определение архитектуры модели и компиляция
input_shape = (128, 128, 3)
num_classes = 6  # Предполагается 6 классов для сегментации

unet_model = build_unet_model(input_shape, num_classes)

# Использование оптимизатора Adagrad с параметром скорости обучения lr=0.01
optimizer = Adagrad(lr=0.01)

unet_model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Обучение модели
history = unet_model.fit(np.array(X_train)/255, Y_train_pred, epochs=10, validation_data=(np.array(X_test)/255, Y_test_pred))


Epoch 1/10
1/1 [==============================] - 15s 15s/step - loss: 1.7664 - accuracy: 0.5758 - val_loss: 1.7568 - val_accuracy: 0.5813
Epoch 2/10
1/1 [==============================] - 12s 12s/step - loss: 1.7586 - accuracy: 0.5944 - val_loss: 1.7493 - val_accuracy: 0.5847
Epoch 3/10
1/1 [==============================] - 12s 12s/step - loss: 1.7510 - accuracy: 0.5991 - val_loss: 1.7421 - val_accuracy: 0.5854
Epoch 4/10
1/1 [==============================] - 12s 12s/step - loss: 1.7437 - accuracy: 0.6003 - val_loss: 1.7353 - val_accuracy: 0.5856
Epoch 5/10
1/1 [==============================] - 12s 12s/step - loss: 1.7368 - accuracy: 0.6007 - val_loss: 1.7287 - val_accuracy: 0.5856
Epoch 6/10
1/1 [==============================] - 12s 12s/step - loss: 1.7302 - accuracy: 0.6008 - val_loss: 1.7223 - val_accuracy: 0.5855
Epoch 7/10
1/1 [==============================] - 12s 12s/step - loss: 1.7237 - accuracy: 0.6007 - val_loss: 1.7159 - val_accuracy: 0.5855
Epoch 8/10
1/1 [===========

# Вывод

На ималых данных удалось добиться только val_accuracy: 0.5855. При изменении оптимизаторов нет существенных изменеий.